# MCB 32L: Lab 04 - Muscle Excitability and Contractility
 
---

### Professor Robin Ball

During this lab, you will get more practice using Python and creating graphs.

*Estimated Time: ~30 mins*

---

### Table of Contents

1. [Entering your data](#section-1)<br>
2. [Graphing your data](#section-2)<br>

In [ ]:
#Run this cell first! Everytime you come back to this notebook, you need to run this cell again.
from datascience import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from table import *
%matplotlib inline
plt.style.use("fivethirtyeight")
grid_options = {"forceFitColumns" : False, "defaultColumnWidth" : 175}

## 1. Entering your data <a id='section-1'></a>
Before you can graph your data, you must enter it into the computer. The cell below will load a spreadsheet with each cell filled with the value NaN .

<div class='alert alert-info'>
    
**QUESTION:** Enter your data from Table 1 into the spreadsheet shown after you run the cell below. Note that if you accidentally run the cell below again, you will lose the data that you previously typed in.

Recall from Lab 0 that you can also make a data table by making a series of lists and then combining them into a table. We think it is easier to enter the data into the grid here, but if you want to make a table from lists there are hints about how to do that below.    
</div>

In [ ]:
# Enter data for subject 1

table_1_1_empty = pd.read_csv("empty-table-1-1.csv")
rows, cols = table_1_1_empty.shape
table_1_1_sheet = make_table(rows, cols, list(table_1_1_empty.columns))

The cell below will take your data and transfer it into a Table for you. Please verify that your data is correct.

In [ ]:
table_1_1 = get_table(table_1_1_sheet)
table_1_1

In [ ]:
#Enter data for subject 2

table_1_2_empty = pd.read_csv("empty-table-1-2.csv")
rows, cols = table_1_2_empty.shape
table_1_2_sheet = make_table(rows, cols, list(table_1_2_empty.columns))

In [ ]:
table_1_2 = get_table(table_1_2_sheet)
table_1_2

Now let's put these tables together!

Just Run the cell below!

In [ ]:
table_1 = merge(table_1_1, table_1_2)
table_1

Only run the cell below if you want to use lists to enter your data into the table. You will need to remove the "#" from the coding lines. Follow the directions in the cell below if you want to use lists.

In [ ]:
# If you prefer to make a table from lists, you can delete the "#" from the next lines
# and enter your numbers in order in the brackets, separated by commas
# Follow the pattern to make lists for the rest of your data

#stim_amp_1 = [...]
#twitch_dur_1 = [...]


# Now make a table from your lists. Enter the name of the column followed by the list you 
# want to put into that column

#table_1 = Table().with_columns("Stimulus Amplitude 1", stim_amp_1, "Twitch Duration 1", twitch_dur_1, ...)

<div class='alert alert-info'>
    
 **QUESTION:** Enter your data from Table 4 into the spreadsheet shown after you run the cell below. 
    
Again, if you prefer to enter the data as lists, follow the directions above but for Table 4. You can always insert a new cell from the Insert menu.     

</div>

In [ ]:
table_4_df = make_table(rows = 5, cols = 2, labels = ["Frequency (Hz)","Twitch Tension"])

The cell below will take your data and transfer it into a Table for you. Please verify that your data is correct.
</div>

In [ ]:
table_4 = get_table(table_4_df)
table_4

## 2. Graphing your data <a id='section-2'></a>

We're interested in a function called `plot`, which makes line graphs. We can tell the computer to find `plot` in the `plt` module using what's called *dot notation*:

In [ ]:
# use the plot function
plt.plot()

`plot` needs to get data to plot as its arguments. In this case, we can give it the lists of data we made earlier. The data goes inside the parentheses: the data that goes on the x-axis is the first argument, and the data that goes on the y-axis is the second.

Here's an example of how to plot Stimulus Amplitude against Twitch Duration for Subject 1. Stimulus Amplitude is on the x-axis, so it goes first in the parentheses.

In [ ]:
# create a line plot of data for subject 1
plt.plot(table_1["Stimulus Amplitude 1"], table_1["Twitch Duration 1"])

# a function to show the line plot
plt.show()

Note how we accessed the values in our table as this will be important later on. To access a column of the table, use the bracket notation (known as "slicing"):

```python
table["Column Name"]
```

The cell below will access the stimulus amplitude for subject 2:

In [ ]:
table_1["Stimulus Amplitude 2"]

As good scientists, we know that this graph is missing a lot. Our axes, graph, and line should all be labeled. Fortunately, the `plt` module has functions to do all of those things. 

Read over the call expressions in the next cell to figure out what each does, then run the cell to make the graph.

<div class="alert alert-warning">

Note that we put labels and titles in quotation marks. This tells the computer that they are text data (also called *strings*). If we didn't put them in quotes, the computer would think they are variables, then throw a fit when it couldn't find what they stood for.

</div>

In [ ]:
# create the line plot for subject 1
plt.plot(table_1["Stimulus Amplitude 1"], table_1["Twitch Duration 1"], label="Subject 1")

# label the x-axis
plt.xlabel("Stimulus amplitude (mA)")

# label the y-axis
plt.ylabel("Twitch duration (ms)")

# title the graph
plt.title("Twitch duration vs. stimulus strength")

# show the legend
plt.legend()

# show the plot
plt.show()

Now it's time to create your graphs!

<div class='alert alert-info'>
    
**QUESTION:** Using the functions you learned in the past and the refresher above, create a line plot of your data. Make one graph from Table 1 of twitch tension vs stimulus amplitude that includes lines for both subjects. Then make another line plot from Table 4 of twitch tension vs the frequency of stimulation.

</div>

In [ ]:
#Graph of table 1, twitch tension vs stimulus amplitude for both subjects 
#Hint: you will need to run the plt.plot function twice, once for each subject

...

In [ ]:
#Graph of table 4, twitch tension vs the frequency of stimulation

...

<div class='alert alert-warning'>
If you are not getting a nice trend line, check that your data has been entered in order of increasing frequency.
</div>

### Saving the Notebook as an html file

Congrats on finishing your lab notebook. To turn in this lab assignment follow the steps below:

>1. Go to the File menu
2. Go to 'Save and Export Notebook As...'
3. Select HTML (.html)
4. Upload your complete notebook in the lab 4 assignment on bCourses.

---
Notebook developed by: Monica Wilkinson and Chris Pyles

Data Science Modules: http://data.berkeley.edu/education/modules
